<a href="https://www.kaggle.com/code/gebrielgidey/compare-attention-ipynb?scriptVersionId=185949079" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#>> updated 4/1
# For viewing and manipulating data
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Importing the necessary libraries
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gensim.downloader as api
from gensim.models import KeyedVectors # >> alternative to gensim.downloader
import matplotlib.pyplot as plt

# Getting particular functions from these libraries
from scipy import stats
from torch import Tensor
from sklearn.utils import resample
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_extraction.text import CountVectorizer
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from collections import Counter

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
reviews_file = ''
w2v_file = ''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        file_name = os.path.join(dirname, filename)
        if file_name.endswith('.csv'): 
            reviews_file = file_name
        elif file_name.endswith('.bin') or ('.gz'):
            w2v_file = file_name
        else:
            print(f'Found unexpected file: {file_name}')
            

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
VERBOSE = True
def printv(text):
    if VERBOSE: print('VERBOSE:', text)
    return

def showV(text):
    '''unconditional verbose output'''
    print('VERBOSE:', text)
    return

DEV = True
def printd(text):
    if DEV: print('DEV:', text)
    return

def showD(text):
    '''unconditional DEV output'''
    print('VERBOSE:', text)
    return

showCellCompletion = False
def showC(text):
    if showCellCompletion:
        print('Cell complete:', text)
    return

import subprocess
showNv = True
accelerator = True

def printNv():
    if not showNv or not accelerator: return
    mem_usage = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE)
    print(mem_usage.stdout.decode('utf-8'))

showMemoryAllocation = True
def printM():
    if not showMemoryAllocation: return
    print(f"Total allocated memory: {torch.cuda.memory_allocated()} bytes")

In [4]:
# Checks if a CUDA enabled GPU is available and prints out its information
if torch.cuda.is_available():
    print("CUDA is available!")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        
    device = torch.device("cuda:0")

else:
    print("CUDA is not available.")
    device = torch.device("cpu")

CUDA is available!
GPU 0: Tesla P100-PCIE-16GB


In [ ]:
# import h5py
# file_path = '/kaggle/input/145000/145000.h5'
# with h5py.File(file_path, 'r') as hf:
#   # Access the datasets within the HDF5 file
#   text_reviews_dataset = hf['text_embeddings']
#   ratings_dataset = hf['rating_labels']
#   masks_dataset = hf['mask']

#   # Convert the datasets to PyTorch tensors
#   text_reviews = torch.from_numpy(text_reviews_dataset[:])
#   ratings = torch.from_numpy(ratings_dataset[:])
#   masks = torch.from_numpy(masks_dataset[:])

# # check loaded tensors

# print(text_reviews.shape)
# print(ratings.shape)
# print(masks.shape)

# class ReviewDataset(Dataset):
#   def __init__(self, text_reviews, ratings, masks):
#     self.text_reviews = text_reviews
#     self.ratings = ratings
#     self.masks = masks

#   def __getitem__(self, index):
#     review = self.text_reviews[index]
#     rating = self.ratings[index]
#     mask = self.masks[index]


#     return review, rating, mask

# print(text_reviews.shape)
# print(ratings.shape)
# print(masks.shape)

# # Create the dataset
# dataset = ReviewDataset(text_reviews, ratings, masks)

# # Create data loaaders
# data_loader = DataLoader(dataset, batch_size = 32, shuffle = False)

# Instantiation

In [5]:
text_embeddings_tensors = torch.rand(1000, 100, 300).to(device)

# Linear projection matrices (takes in 300 dimesions, outputs 300 dimensions)
W_k = nn.Linear(300, 300).to(device)
W_q = nn.Linear(300, 300).to(device)
W_v = nn.Linear(300, 300).to(device)

# Compute key, query, and value tensors
key_tensor   = W_k(text_embeddings_tensors) # shape: (batch_size, max_len, emb)
query_tensor = W_q(text_embeddings_tensors) # shape: (batch_size, max_len, emb)
value_tensor = W_v(text_embeddings_tensors) # shape: (batch_size, max_len, emb)

print(text_embeddings_tensors[0])
print(key_tensor[0])
print(query_tensor[0])
print(value_tensor[0])

tensor([[0.5121, 0.3123, 0.1569,  ..., 0.3187, 0.2253, 0.9384],
        [0.6458, 0.8690, 0.7288,  ..., 0.1773, 0.5933, 0.5932],
        [0.8236, 0.8649, 0.6571,  ..., 0.1761, 0.2031, 0.9777],
        ...,
        [0.4672, 0.2094, 0.0991,  ..., 0.7459, 0.8336, 0.9345],
        [0.7825, 0.9908, 0.7685,  ..., 0.7729, 0.5302, 0.3785],
        [0.1011, 0.3027, 0.8615,  ..., 0.2218, 0.2238, 0.0121]],
       device='cuda:0')
tensor([[ 2.4702e-02,  4.6389e-01, -8.7308e-02,  ...,  4.1977e-01,
          2.4056e-01,  4.2406e-01],
        [ 1.1506e-01,  5.1160e-01, -3.0963e-02,  ...,  2.4979e-01,
          3.2743e-01,  2.3399e-02],
        [ 2.8117e-01,  5.9181e-01,  9.6312e-05,  ...,  2.4495e-01,
          7.2934e-02,  3.0909e-01],
        ...,
        [ 8.8383e-03,  5.3434e-01,  2.7508e-03,  ...,  3.1964e-01,
          1.8849e-01,  1.7927e-01],
        [-2.1820e-01,  1.5770e-01,  2.6884e-02,  ...,  2.0175e-01,
          4.4826e-01,  2.6815e-01],
        [-9.2963e-02,  2.4232e-01,  9.9712e-02,  .

# Hyperparams

In [6]:
# HyperParameters for the module
d_model = 300  # Should match the embedding dimension of our word embeddings
seq_len = 100  # Maximum sequence length
input_size = d_model  # based on the output size of our feed-forward network

num_layers = 1 # 4 # Number of encoder layers
h       = 1 # 10   # number of attention head
d_ffn   = 2048 # dimension of the feedforward layer

dropout = 0.0 # 0.1  # we can adjust the dropout if needed
eps     = 1e-05 # epsilon value to prevent the standard deviation from becoming zero
learning_rate = 0.01

"""
To ensure compatibility, it's important to choose the 
number of attention heads (h) such that d_model is 
evenly divisible by h in the multi-head attention 
module's self.d_k. This allows for a clean distribution 
of the model dimensionality across the attention heads.
"""

# d_model / attn.h = 300 / 10 = 30

"\nTo ensure compatibility, it's important to choose the \nnumber of attention heads (h) such that d_model is \nevenly divisible by h in the multi-head attention \nmodule's self.d_k. This allows for a clean distribution \nof the model dimensionality across the attention heads.\n"

#### Our Scaled Dot Product Attention 
#### Our Multi-Head Attention

#### -------------------------------------------------------

#### Pytorch's Scaled Dot Product Attention
#### Pytorch's Multi-Head Attention

#### -------------------------------------------------------

#### Our Transformer Encoder
#### Pytorch's Transformer Encoder
#### Pytorch's Transformer Encoder (Custom)

# Our Scaled Dot Product Attention 
# Our Multi-Head Attention

### Mute our tensor generators and use tensors from above implementation

*In this modified version, we've removed the linear layers for projecting the input embeddings to query, key, and value tensors. Instead, the forward method now accepts pre-computed query, key, and value tensors as input.*

In [7]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h: int, d_model: int, dropout: float = 0.0):
        super().__init__()

        self.d_model = d_model
        self.h = h
        self.d_k = d_model // h

        self.query_linear = nn.Linear(d_model, d_model)
        self.key_linear = nn.Linear(d_model, d_model)
        self.value_linear = nn.Linear(d_model, d_model)
        self.output_linear = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(p=dropout)

    @staticmethod
    def scaled_dot_product_attention(query, key, value, mask=None, dropout=None):
        """
        Compute the scaled dot-product attention.

        Args:
            query: Tensor of shape [batch_size, h, seq_len, d_k].
            key: Tensor of shape [batch_size, h, seq_len, d_k].
            value: Tensor of shape [batch_size, h, seq_len, d_k].
            mask: Optional mask tensor of shape [batch_size, 1, seq_len, seq_len].
            dropout: Optional dropout layer.

        Returns:
            attention_output: Tensor of shape [batch_size, h, seq_len, d_k].
            attention_weights: Tensor of shape [batch_size, h, seq_len, seq_len].
        """
        d_k = query.size(-1)
        attention_scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)

        if mask is not None:
            # Ensure mask has the same shape as attention_scores
            mask = mask.unsqueeze(1)  # Add dimension for heads first
            # Then expand to match attention_scores
            mask = mask.expand(-1, query.size(1), -1, -1)
            attention_scores = attention_scores.masked_fill(mask == 0, float('-inf'))

        attention_weights = torch.softmax(attention_scores, dim=-1)

        if dropout is not None:
            attention_weights = dropout(attention_weights)

        attention_output = torch.matmul(attention_weights, value)

        return attention_output, attention_weights

    def forward(self, query, key, value, mask=None):
        """
        Compute the multi-headed attention.

        Args:
            query: Tensor of shape [batch_size, seq_len, d_model].
            key: Tensor of shape [batch_size, seq_len, d_model].
            value: Tensor of shape [batch_size, seq_len, d_model].
            mask: Optional mask tensor of shape [batch_size, seq_len, seq_len].

        Returns:
            output: Tensor of shape [batch_size, seq_len, d_model].
        """
        batch_size = query.size(0)

#         query = self.query_linear(query).view(batch_size, -1, self.h, self.d_k).transpose(1, 2)
#         key = self.key_linear(key).view(batch_size, -1, self.h, self.d_k).transpose(1, 2)
#         value = self.value_linear(value).view(batch_size, -1, self.h, self.d_k).transpose(1, 2)

        if mask is not None:
            mask = mask.unsqueeze(1)  # Add a dimension for h

        # attention_output, _ = self.scaled_dot_product_attention(query, key, value, mask, self.dropout)
        attention_output, attention_weights = self.scaled_dot_product_attention(query, key, value, mask, self.dropout) 

        attention_output = attention_output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)

        output = self.output_linear(attention_output)

        #return output
        return attention_weights, output # Return both values
    
# ours
our_mha = MultiHeadedAttention(h, d_model, dropout).to(device)

# Use pre-computed query, key, and value tensors
our_attn_weights, our_attn_output = our_mha(query_tensor, key_tensor, value_tensor)

print(our_attn_weights[0])
print(our_attn_output[0])

print("Attention Weights (shape:", our_attn_weights.shape, ")")
print("Attention Scores (shape:", our_attn_output.shape, ")")

tensor([[0.0103, 0.0095, 0.0098,  ..., 0.0097, 0.0109, 0.0096],
        [0.0101, 0.0094, 0.0104,  ..., 0.0101, 0.0103, 0.0098],
        [0.0099, 0.0094, 0.0107,  ..., 0.0098, 0.0100, 0.0092],
        ...,
        [0.0107, 0.0099, 0.0104,  ..., 0.0097, 0.0097, 0.0098],
        [0.0101, 0.0094, 0.0097,  ..., 0.0098, 0.0102, 0.0093],
        [0.0109, 0.0095, 0.0099,  ..., 0.0106, 0.0097, 0.0094]],
       device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[ 0.1800,  0.1482, -0.0772,  ...,  0.1646, -0.0510,  0.1309],
        [ 0.1610,  0.0924, -0.1067,  ...,  0.0653, -0.1533,  0.1574],
        [-0.0397,  0.0012,  0.0203,  ..., -0.0285, -0.0270, -0.0785],
        ...,
        [ 0.0859,  0.0469, -0.0650,  ...,  0.0068, -0.1386,  0.0817],
        [-0.1329, -0.0972, -0.2016,  ..., -0.0973,  0.1515, -0.1282],
        [-0.1302, -0.0847,  0.0308,  ..., -0.1535, -0.0661, -0.1399]],
       device='cuda:0', grad_fn=<SelectBackward0>)
Attention Weights (shape: torch.Size([1000, 100, 100]) )
Attention

# Pytorch's Scaled Dot Product Attention

In [8]:
# FROM TORCH.NN.FUNCTIONAL.SCALED_DOT_PRODUCT_ATTENTION DOCS

# Efficient implementation equivalent to the following:
def scaled_dot_product_attention(query, key, value, attn_mask=None, dropout_p=0.0, is_causal=False, scale=None) -> torch.Tensor:
    L, S = query.size(-2), key.size(-2)
    scale_factor = 1 / math.sqrt(query.size(-1)) if scale is None else scale
    attn_bias = torch.zeros(L, S, dtype=query.dtype, device = device)
    if is_causal:
        assert attn_mask is None
        temp_mask = torch.ones(L, S, dtype=torch.bool).tril(diagonal=0)
        attn_bias.masked_fill_(temp_mask.logical_not(), float("-inf"))
        attn_bias.to(query.dtype)

    if attn_mask is not None:
        if attn_mask.dtype == torch.bool:
            attn_bias.masked_fill_(attn_mask.logical_not(), float("-inf"))
        else:
            attn_bias += attn_mask
    attn_weight = query @ key.transpose(-2, -1) * scale_factor
    attn_weight += attn_bias
    attn_weight = torch.softmax(attn_weight, dim=-1)
    attn_weight = torch.dropout(attn_weight, dropout_p, train=True)
    return attn_weight, attn_weight @ value

# pytorch
attn_weights, attn_output = scaled_dot_product_attention(
    query_tensor, key_tensor, value_tensor)

print(attn_weights[0])
print(attn_output[0])

print("Attention Weights (shape:", attn_weights.shape, ")")
print("Attention Scores (shape:", attn_output.shape, ")")

tensor([[0.0103, 0.0095, 0.0098,  ..., 0.0097, 0.0109, 0.0096],
        [0.0101, 0.0094, 0.0104,  ..., 0.0101, 0.0103, 0.0098],
        [0.0099, 0.0094, 0.0107,  ..., 0.0098, 0.0100, 0.0092],
        ...,
        [0.0107, 0.0099, 0.0104,  ..., 0.0097, 0.0097, 0.0098],
        [0.0101, 0.0094, 0.0097,  ..., 0.0098, 0.0102, 0.0093],
        [0.0109, 0.0095, 0.0099,  ..., 0.0106, 0.0097, 0.0094]],
       device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[ 0.2117,  0.0447,  0.2545,  ..., -0.2009,  0.0257, -0.0692],
        [ 0.2114,  0.0458,  0.2538,  ..., -0.2007,  0.0253, -0.0685],
        [ 0.2109,  0.0454,  0.2545,  ..., -0.2003,  0.0248, -0.0696],
        ...,
        [ 0.2103,  0.0452,  0.2550,  ..., -0.1998,  0.0247, -0.0687],
        [ 0.2112,  0.0457,  0.2545,  ..., -0.2005,  0.0258, -0.0689],
        [ 0.2112,  0.0450,  0.2542,  ..., -0.2001,  0.0251, -0.0685]],
       device='cuda:0', grad_fn=<SelectBackward0>)
Attention Weights (shape: torch.Size([1000, 100, 100]) )
Attention

# Pytorch's Multi-Head Attention

In [9]:
multihead_attn = nn.MultiheadAttention(d_model, h).to(device)
multi_attn_output, multi_attn_weights = multihead_attn(
    query_tensor, key_tensor, value_tensor)

print(multi_attn_weights[0])
print(multi_attn_output[0])

print("Attention Weights (shape:", multi_attn_weights.shape, ")")
print("Attention Scores (shape:", multi_attn_output.shape, ")")

tensor([[0.0010, 0.0010, 0.0010,  ..., 0.0010, 0.0010, 0.0010],
        [0.0010, 0.0010, 0.0010,  ..., 0.0010, 0.0010, 0.0010],
        [0.0010, 0.0010, 0.0010,  ..., 0.0010, 0.0009, 0.0010],
        ...,
        [0.0010, 0.0010, 0.0010,  ..., 0.0010, 0.0010, 0.0010],
        [0.0010, 0.0010, 0.0010,  ..., 0.0010, 0.0010, 0.0010],
        [0.0010, 0.0010, 0.0010,  ..., 0.0010, 0.0010, 0.0010]],
       device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[ 0.0044, -0.0584,  0.2606,  ..., -0.1229, -0.0265, -0.0580],
        [ 0.0013, -0.0609,  0.2588,  ..., -0.1209, -0.0279, -0.0578],
        [ 0.0051, -0.0573,  0.2582,  ..., -0.1231, -0.0282, -0.0585],
        ...,
        [ 0.0067, -0.0647,  0.2595,  ..., -0.1183, -0.0284, -0.0589],
        [ 0.0035, -0.0604,  0.2568,  ..., -0.1192, -0.0294, -0.0575],
        [ 0.0053, -0.0592,  0.2605,  ..., -0.1220, -0.0273, -0.0556]],
       device='cuda:0', grad_fn=<SelectBackward0>)
Attention Weights (shape: torch.Size([100, 1000, 1000]) )
Attentio

# Our Transformer Encoder

In [10]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h: int, d_model: int, dropout: float = 0.0):
        super().__init__()

        self.d_model = d_model
        self.h = h
        self.d_k = d_model // h

        self.query_linear = nn.Linear(d_model, d_model)
        self.key_linear = nn.Linear(d_model, d_model)
        self.value_linear = nn.Linear(d_model, d_model)
        self.output_linear = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(p=dropout)

    @staticmethod
    def scaled_dot_product_attention(query, key, value, mask=None, dropout=None):
        """
        Compute the scaled dot-product attention.

        Args:
            query: Tensor of shape [batch_size, h, seq_len, d_k].
            key: Tensor of shape [batch_size, h, seq_len, d_k].
            value: Tensor of shape [batch_size, h, seq_len, d_k].
            mask: Optional mask tensor of shape [batch_size, 1, seq_len, seq_len].
            dropout: Optional dropout layer.

        Returns:
            attention_output: Tensor of shape [batch_size, h, seq_len, d_k].
            attention_weights: Tensor of shape [batch_size, h, seq_len, seq_len].
        """
        d_k = query.size(-1)
        attention_scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)

        if mask is not None:
            # Ensure mask has the same shape as attention_scores
            mask = mask.unsqueeze(1)  # Add dimension for heads first
            # Then expand to match attention_scores
            mask = mask.expand(-1, query.size(1), -1, -1)
            attention_scores = attention_scores.masked_fill(mask == 0, float('-inf'))

        attention_weights = torch.softmax(attention_scores, dim=-1)

        if dropout is not None:
            attention_weights = dropout(attention_weights)

        attention_output = torch.matmul(attention_weights, value)

        return attention_output, attention_weights

    def forward(self, query, key, value, mask=None):
        """
        Compute the multi-headed attention.

        Args:
            query: Tensor of shape [batch_size, seq_len, d_model].
            key: Tensor of shape [batch_size, seq_len, d_model].
            value: Tensor of shape [batch_size, seq_len, d_model].
            mask: Optional mask tensor of shape [batch_size, seq_len, seq_len].

        Returns:
            output: Tensor of shape [batch_size, seq_len, d_model].
        """
        batch_size = query.size(0)

#         query = self.query_linear(query).view(batch_size, -1, self.h, self.d_k).transpose(1, 2)
#         key = self.key_linear(key).view(batch_size, -1, self.h, self.d_k).transpose(1, 2)
#         value = self.value_linear(value).view(batch_size, -1, self.h, self.d_k).transpose(1, 2)

        if mask is not None:
            mask = mask.unsqueeze(1)  # Add a dimension for h

        # attention_output, _ = self.scaled_dot_product_attention(query, key, value, mask, self.dropout)
        attention_output, attention_weights = self.scaled_dot_product_attention(query, key, value, mask, self.dropout) 

        attention_output = attention_output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)

        output = self.output_linear(attention_output)

        #return output
        return attention_weights, output # Return both values

class LayerNorm(nn.Module):
    def __init__(self, features: int, eps: float = 1e-6):
        """
        Construct a layernorm module.

        Args:
            features: Number of input features.
            eps: Small value added to the denominator for numerical stability.
        """
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        """
        Apply layer normalization.

        Args:
            x: Input tensor of shape [batch_size, seq_len, features].

        Returns:
            Normalized tensor of the same shape as the input.
        """
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model: int, d_ffn: int, dropout: float):
        """
        Construct a position-wise feed-forward network.

        Args:
            d_model: Dimension of the input embeddings.
            d_ffn: Dimension of the feed-forward network.
            dropout: Probability of dropout.
        """
        super().__init__()
        self.w_1 = nn.Linear(d_model, d_ffn)
        self.w_2 = nn.Linear(d_ffn, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        Apply the position-wise feed-forward network.

        Args:
            x: Input tensor of shape [batch_size, seq_len, d_model].

        Returns:
            Output tensor of shape [batch_size, seq_len, d_model].
        """
        return self.w_2(self.dropout(torch.relu(self.w_1(x))))

class EncoderLayer(nn.Module):
    def __init__(self, d_model: int, h: int, d_ffn: int, dropout: float):
        """
        Construct an encoder layer.

        Args:
            d_model: Dimension of the input embeddings.
            h: Number of attention heads.
            d_ffn: Dimension of the feed-forward network.
            dropout: Probability of dropout.
        """
        super().__init__()
        self.self_attn = MultiHeadedAttention(h, d_model, dropout)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ffn, dropout)
        self.sublayer = nn.ModuleList([LayerNorm(d_model) for _ in range(2)])
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        """
        Apply the encoder layer.

        Args:
            x: Input tensor of shape [batch_size, seq_len, d_model].
            mask: Attention mask tensor of shape [batch_size, seq_len, seq_len].

        Returns:
            Output tensor of shape [batch_size, seq_len, d_model].
        """
        sublayer_output = self.self_attn(x, x, x, mask)
        x = x + self.dropout(sublayer_output)
        x = self.sublayer[0](x)
        sublayer_output = self.feed_forward(x)
        x = x + self.dropout(sublayer_output)
        x = self.sublayer[1](x)
        return x

class StackedEncoder(nn.Module):
    def __init__(self, num_layers: int, d_model: int, h: int, d_ffn: int, dropout: float):
        super().__init__()
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, h, d_ffn, dropout) for _ in range(num_layers)
        ])
        self.norm = LayerNorm(d_model)

    def forward(self, key, query, value, mask=None):
        """
        Apply the stacked encoder.

        Args:
            key: Input tensor of shape [batch_size, seq_len, d_model].
            query: Input tensor of shape [batch_size, seq_len, d_model].
            value: Input tensor of shape [batch_size, seq_len, d_model].
            mask: Attention mask tensor of shape [batch_size, seq_len, seq_len].

        Returns:
            tuple: (attention_weights, output)
                attention_weights: Tensor of shape [batch_size, num_layers, h, seq_len, seq_len].
                output: Tensor of shape [batch_size, seq_len, d_model].
        """
        attention_weights = []
        x = query  # Start with query as the input

        for layer in self.layers:
            layer_weights, x = layer(key, query, value, mask)
            attention_weights.append(layer_weights)

        x = self.norm(x)
        attention_weights = torch.stack(attention_weights, dim=1)
        
        return attention_weights, x

# Update the EncoderLayer to return attention weights
class EncoderLayer(nn.Module):
    def __init__(self, d_model: int, h: int, d_ffn: int, dropout: float):
        super().__init__()
        self.self_attn = MultiHeadedAttention(h, d_model, dropout)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ffn, dropout)
        self.sublayer = nn.ModuleList([LayerNorm(d_model) for _ in range(2)])
        self.dropout = nn.Dropout(dropout)

    def forward(self, key, query, value, mask=None):
        attn_weights, sublayer_output = self.self_attn(query, key, value, mask)
        x = query + self.dropout(sublayer_output)
        x = self.sublayer[0](x)
        sublayer_output = self.feed_forward(x)
        x = x + self.dropout(sublayer_output)
        x = self.sublayer[1](x)
        return attn_weights, x

# Usage
stacked_encoder = StackedEncoder(num_layers, d_model, h, d_ffn, dropout).to(device)

# When using the encoder
our_encoder_weights, our_encoder_output = stacked_encoder(key_tensor, query_tensor, value_tensor, mask=None)

print(our_encoder_weights[0])
print(our_encoder_output[0])

print("Attention Weights (shape:", our_encoder_weights.shape, ")")
print("Encoder Output (shape:", our_encoder_output.shape, ")")

tensor([[[0.0103, 0.0095, 0.0098,  ..., 0.0097, 0.0109, 0.0096],
         [0.0101, 0.0094, 0.0104,  ..., 0.0101, 0.0103, 0.0098],
         [0.0099, 0.0094, 0.0107,  ..., 0.0098, 0.0100, 0.0092],
         ...,
         [0.0107, 0.0099, 0.0104,  ..., 0.0097, 0.0097, 0.0098],
         [0.0101, 0.0094, 0.0097,  ..., 0.0098, 0.0102, 0.0093],
         [0.0109, 0.0095, 0.0099,  ..., 0.0106, 0.0097, 0.0094]]],
       device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[ 0.0590, -0.7327, -0.9108,  ..., -0.8204, -0.1390, -0.5511],
        [-0.4166, -1.2057, -1.3338,  ..., -0.7905, -0.3517, -0.2234],
        [-1.0198,  0.3016, -1.0658,  ..., -1.0742, -0.1294,  0.0723],
        ...,
        [-0.1843, -0.0950, -1.4184,  ..., -0.6220, -1.1983,  0.6643],
        [-0.7517, -0.9233, -0.7616,  ...,  0.0027,  0.0032,  0.5945],
        [-1.3442, -0.7566, -1.2242,  ..., -0.5650, -0.0825,  0.2474]],
       device='cuda:0', grad_fn=<SelectBackward0>)
Attention Weights (shape: torch.Size([1000, 1, 100, 100]) 

238

In [11]:
encoder_layer = nn.TransformerEncoderLayer(d_model, h, d_ffn, dropout).to(device)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers).to(device)

pytorch_encoder_weights, pytorch_encoder_output = transformer_encoder(key_tensor, query_tensor, value_tensor, mask=None)

print(pytorch_encoder_weights[0])
print(pytorch_encoder_output[0])

print("Attention Weights (shape:", pytorch_encoder_weights.shape, ")")
print("Encoder Output (shape:", pytorch_encoder_output.shape, ")")

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


TypeError: TransformerEncoder.forward() got multiple values for argument 'mask'

# Pytorch's Transformer Encoder (Custom)

In [12]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.dropout = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, query, key, value, src_mask=None):
        """
        Pass the input through self-attention, feed-forward, and layer normalization.

        Args:
          src: Tensor - The input sequence (batch_size, seq_len, d_model).
          src_mask: Tensor (optional) - Mask for the source sequence (batch_size, 1, 1, seq_len).

        Returns:
          Tuple[Tensor, Tensor] - The processed output (batch_size, seq_len, d_model) 
                                  and self-attention weights (batch_size, nheads, seq_len, seq_len).
        """
        # Get self-attention weights along with the output
        src, attn_weights = self.self_attn(query, key, value, attn_mask=src_mask)
        src = src + src  # Add & Residual Connection

        src = self.norm1(src)  # LayerNorm before self-attention
        src = self.dropout(src)  # Dropout after self-attention

        src = self.linear2(self.dropout(self.linear1(src)))
        src = src + src  # Add & Residual Connection after dropout
        src = self.norm2(src)  # LayerNorm before feed-forward

        return src, attn_weights
    
custom_pytorch_transformer = TransformerEncoderLayer(d_model, h, d_ffn, dropout).to(device)
custom_pytorch_output, custom_pytorch_output_weights = custom_pytorch_transformer(query_tensor, key_tensor, value_tensor)

print(custom_pytorch_output_weights[0])
print(custom_pytorch_output[0])

print("Attention Weights (shape:", custom_pytorch_output_weights.shape, ")")
print("Attention Scores (shape:", custom_pytorch_output.shape, ")")

tensor([[0.0010, 0.0010, 0.0010,  ..., 0.0010, 0.0010, 0.0010],
        [0.0010, 0.0010, 0.0010,  ..., 0.0010, 0.0010, 0.0010],
        [0.0010, 0.0010, 0.0010,  ..., 0.0010, 0.0010, 0.0010],
        ...,
        [0.0010, 0.0010, 0.0010,  ..., 0.0010, 0.0010, 0.0010],
        [0.0010, 0.0010, 0.0010,  ..., 0.0010, 0.0010, 0.0010],
        [0.0010, 0.0010, 0.0010,  ..., 0.0010, 0.0010, 0.0009]],
       device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[ 1.5555,  1.4374,  1.5129,  ...,  0.5679, -2.8997,  1.0194],
        [ 1.5881,  1.4268,  1.5469,  ...,  0.5640, -2.8892,  1.0321],
        [ 1.6105,  1.4418,  1.5330,  ...,  0.6094, -2.9029,  1.0125],
        ...,
        [ 1.5670,  1.4322,  1.5513,  ...,  0.5765, -2.8570,  1.0492],
        [ 1.5707,  1.4129,  1.5393,  ...,  0.5673, -2.8711,  1.0418],
        [ 1.5739,  1.4145,  1.5430,  ...,  0.6067, -2.8735,  1.0201]],
       device='cuda:0', grad_fn=<SelectBackward0>)
Attention Weights (shape: torch.Size([100, 1000, 1000]) )
Attentio

# Comparison

In [20]:
# Our Scaled Dot Product Attention / Our Multi-Head Attention
print(our_attn_weights[0])
print(our_attn_output[0])

# Pytorch's Scaled Dot Product Attention
print(attn_weights[0])
print(attn_output[0])

# Pytorch's Multi-Head Attention
print(multi_attn_weights[0])
print(multi_attn_output[0])

# Our Transformer Encoder
print(our_encoder_weights[0])
print(our_encoder_output[0])

# Pytorch's Transformer Encoder
# print(pytorch_encoder_weights[0])
# print(pytorch_encoder_output[0])

# Pytorch's Transformer Encoder (Custom)
print(custom_pytorch_output_weights[0])
print(custom_pytorch_output[0])

tensor([[0.0104, 0.0101, 0.0099,  ..., 0.0088, 0.0103, 0.0100],
        [0.0105, 0.0102, 0.0097,  ..., 0.0092, 0.0109, 0.0099],
        [0.0103, 0.0104, 0.0091,  ..., 0.0093, 0.0110, 0.0102],
        ...,
        [0.0102, 0.0101, 0.0087,  ..., 0.0094, 0.0104, 0.0099],
        [0.0102, 0.0100, 0.0095,  ..., 0.0087, 0.0107, 0.0100],
        [0.0108, 0.0101, 0.0093,  ..., 0.0090, 0.0102, 0.0104]],
       device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[0.0104, 0.0101, 0.0099,  ..., 0.0088, 0.0103, 0.0100],
        [0.0105, 0.0102, 0.0097,  ..., 0.0092, 0.0109, 0.0099],
        [0.0103, 0.0104, 0.0091,  ..., 0.0093, 0.0110, 0.0102],
        ...,
        [0.0102, 0.0101, 0.0087,  ..., 0.0094, 0.0104, 0.0099],
        [0.0102, 0.0100, 0.0095,  ..., 0.0087, 0.0107, 0.0100],
        [0.0108, 0.0101, 0.0093,  ..., 0.0090, 0.0102, 0.0104]],
       device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[0.0010, 0.0010, 0.0010,  ..., 0.0010, 0.0010, 0.0010],
        [0.0010, 0.0010, 0.0010,  ...,

In [24]:
def cosine_similarity(tensor1, tensor2):
    tensor1_norm = tensor1 / tensor1.norm(dim=-1, keepdim=True)
    tensor2_norm = tensor2 / tensor2.norm(dim=-1, keepdim=True)
    return (tensor1_norm * tensor2_norm).sum(dim=-1)

# cosine, mse, and correlation
similarity = cosine_similarity(custom_pytorch_output_weights, our_encoder_weights)
mse = torch.mean((custom_pytorch_output_weights - our_encoder_weights) ** 2)
correlation = torch.corrcoef(torch.stack((custom_pytorch_output_weights.view(-1), our_encoder_weights.view(-1))))

print("Cosine Similarity:", similarity)
print("Mean Squared Error:", mse)
print("Correlation Coefficient:", correlation[0, 1])

RuntimeError: The size of tensor a (1000) must match the size of tensor b (100) at non-singleton dimension 3

In [ ]:
# def compare_similarities(attn_weights, multi_attn_output_weights):
#     # Cosine similarity with multihead attention weights
#     multihead_similarity = cosine_similarity(our_attn_weights, multi_attn_output_weights)
#     # MSE with attn_weights and multihead attention weights
#     mse_attn_multihead = torch.mean((our_attn_weights - multi_attn_output_weights) ** 2)
#     # Correlation with attn_weights and our_attn_weights (reshape for corrcoef)
#     correlation = torch.corrcoef(torch.stack((our_attn_weights.view(-1), multi_attn_output_weights.view(-1))))

#     print("Cosine Similarity (Attn vs Multihead):", multihead_similarity)
#     print("Mean Squared Error (Attn vs Multihead):", mse_attn_multihead)
#     print("Correlation Coefficient:", correlation[0, 1])

# # Usage example
# compare_similarities(attn_weights, multi_attn_output_weights)

# Visualize

In [ ]:
# pytorch

def plot_attention_map(attention_weights, labels = None, figsize = (8, 8), dpi = 100):
    """
    Plots an attention map given the attention weights and optional labels.

    Args:
        attention_weights (torch.Tensor): A square matrix of attention weights.
        labels (list, optional): A list of labels for the tokens. Defaults to None.
        figsize (tuple, optional): The figure size in inches. Defaults to (8, 8).
        dpi (int, optional): The resolution of the figure in dots per inch. Defaults to 100.

    Returns:
        None
    """
    fig, ax = plt.subplots(figsize = figsize, dpi = dpi)
    # remove detach
    im = ax.imshow(attention_weights.cpu().detach().numpy(), cmap = 'viridis')

    if labels:
        step = max(len(labels) // 10, 1)  # Adjust tick frequency based on the number of labels
        ax.set_xticks(range(0, len(labels), step))
        ax.set_yticks(range(0, len(labels), step))
        ax.set_xticklabels(labels[::step], rotation = 90)
        ax.set_yticklabels(labels[::step])

    ax.set_xlabel('Key')
    ax.set_ylabel('Query')
    ax.set_title('Attention Map')
 
    cbar = ax.figure.colorbar(im, ax = ax)
    cbar.ax.set_ylabel('Attention Weight', rotation = -90, va = 'bottom')

    fig.tight_layout()
    plt.show()

# Replace with your actual token labels
labels = ['Token {}'.format(i) for i in range(seq_len)]
plot_attention_map(attn_weights[0], labels)

# ours

def plot_attention_map(attention_weights, labels = None, figsize = (8, 8), dpi = 100):
    """
    Plots an attention map given the attention weights and optional labels.

    Args:
        attention_weights (torch.Tensor): A square matrix of attention weights.
        labels (list, optional): A list of labels for the tokens. Defaults to None.
        figsize (tuple, optional): The figure size in inches. Defaults to (8, 8).
        dpi (int, optional): The resolution of the figure in dots per inch. Defaults to 100.

    Returns:
        None
    """
    fig, ax = plt.subplots(figsize = figsize, dpi = dpi)
    # remove detach
    im = ax.imshow(attention_weights.cpu().detach().numpy(), cmap = 'viridis')

    if labels:
        step = max(len(labels) // 10, 1)  # Adjust tick frequency based on the number of labels
        ax.set_xticks(range(0, len(labels), step))
        ax.set_yticks(range(0, len(labels), step))
        ax.set_xticklabels(labels[::step], rotation = 90)
        ax.set_yticklabels(labels[::step])

    ax.set_xlabel('Key')
    ax.set_ylabel('Query')
    ax.set_title('Attention Map')
 
    cbar = ax.figure.colorbar(im, ax = ax)
    cbar.ax.set_ylabel('Attention Weight', rotation = -90, va = 'bottom')

    fig.tight_layout()
    plt.show()

# Replace with your actual token labels
labels = ['Token {}'.format(i) for i in range(seq_len)]
plot_attention_map(our_attn_weights[0], labels)

*Comparing the attention outputs of our scaled dot product function and PyTorch's function is almost similar to comparing the attention outputs of different heads in a multi-head attention system. The differences arise from factors such as random initialization, implementation variations, and numerical precision, but the fundamental principles and functionality of the attention mechanism remain consistent.*

*Random Initialization: The weights of the linear layers used for query, key, and value projections are randomly initialized in both functions. This random initialization leads to different starting points and can result in different attention patterns.*

*Implementation Differences: Although both functions aim to implement the scaled dot product attention mechanism, there might be slight differences in their implementations, such as the order of operations, the use of specific PyTorch functions, or the handling of edge cases. These differences can contribute to variations in the attention outputs.*

*Numerical Precision: The attention outputs can be sensitive to numerical precision, especially when dealing with large input sequences or high-dimensional representations. Differences in numerical precision between our function and PyTorch's function can lead to slight variations in the attention outputs.*